In [12]:

## Import Carla and other packages
import carla 
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm
import glob
import os
import sys
import argparse
from datetime import datetime
import random
import pickle



In [13]:

# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [14]:
#Unload all the unwaned things from the world

world.unload_map_layer(carla.MapLayer.Foliage)
world.unload_map_layer(carla.MapLayer.Buildings)
world.unload_map_layer(carla.MapLayer.Decals)
world.unload_map_layer(carla.MapLayer.ParkedVehicles)
world.unload_map_layer(carla.MapLayer.Props)
world.unload_map_layer(carla.MapLayer.StreetLights)
world.unload_map_layer(carla.MapLayer.Walls)
world.unload_map_layer(carla.MapLayer.Particles)
world.load_map_layer(carla.MapLayer.Ground)

In [15]:
##Selecting a spawn point for the ego vehicle
spawnPoint=carla.Transform(carla.Location(x=0,y=0, z=0.598),carla.Rotation(pitch=0.0, yaw=0.0, roll=0.000000))
#Spawn Ego vehicle
Ego_Vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
Ego_Vehicle = world.try_spawn_actor(Ego_Vehicle_bp, spawnPoint)

# Move spectator to view ego vehicle
spectator = world.get_spectator() 
transform = carla.Transform(Ego_Vehicle.get_transform().transform(carla.Location(x=-25,z=10,y=0)),Ego_Vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [16]:
##Selecting a random spawn point for the target vehicle
spawnPoint1=carla.Transform(carla.Location(x=10,y=0, z=0.598),carla.Rotation(pitch=0.0, yaw=0.0, roll=0.000000))
#Spawn Target vehicle
Target_Vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
Target_Vehicle = world.try_spawn_actor(Target_Vehicle_bp, spawnPoint1)


In [ ]:
#Target_Vehicle.destroy()

In [ ]:
##get the bounding box location of Target Vehicle
"""
box1 = Target_Vehicle.bounding_box
print(box.get_local_vertices)         # Location relative to the vehicle.
print(box.extent) 
print(spawnPoint.location)  
print(spawnPoint) 
print(carla.Transform(box.location))       # XYZ half-box extents in meters.
if carla.BoundingBox.contains(box,spawnPoint.location,carla.Transform(box.location)):
    print('True')
"""

In [ ]:
for actor in world.get_actors().filter('*lidar*'):
    actor.destroy()

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [17]:
def process_lidar_sematic(point_cloud):
    # data.save_to_disk(parent_path + 'semantic-%06d.ply' % data.frame_number)
    parent_path='/home/santhanam.17/Carla_scripts/point/'

    #Extracting the raw data
    data = np.frombuffer(point_cloud.raw_data, dtype=np.dtype([
        ('x', np.float32), ('y', np.float32), ('z', np.float32),
        ('CosAngle', np.float32), ('ObjIdx', np.uint32), ('ObjTag', np.uint32)]))
    
    data_updated=data[data['ObjTag']==10]
    
    points_temp = np.array([data_updated['x'], data_updated['y'], data_updated['z']]).T
    point_list=o3d.geometry.PointCloud()
    point_list.points = o3d.utility.Vector3dVector(points_temp)
    o3d.io.write_point_cloud("/home/santhanam.17/Carla_scripts/point/"+str(point_cloud.frame)+".ply", point_list)

    #carla.LidarMeasurement.save_to_disk(points_new,"/home/santhanam.17/Carla_scripts/point/a.ply")
    #print(points)



visualize Vehicle point cloud using pcl -- next step()
removal of dupilcate points
Lidar Azimuth angle data (*position") - Sloution C
o3d to 
ros publish temporary -Solution B Velodyne64
Full point 
Work on ros node publish, and subscribe random query point
merge point cloud  and publish 
capture rosbag and send the rosnode 

Generate table 

topic 1: background
topic 2: query position 6 dof
topic 3: combined pcl

Algorithm to find the Manhattan distance between the points!


visualize using rviz


In [18]:
def start_lidar():
    # --------------
    # Add a new LIDAR sensor to my ego
    lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast_semantic')
    lidar_bp.set_attribute('channels',str(64))
    lidar_bp.set_attribute('points_per_second',str(1280000))
    lidar_bp.set_attribute('rotation_frequency',str(10))
    lidar_bp.set_attribute('range',str(120))
    lidar_bp.set_attribute('upper_fov',str(0))
    lidar_bp.set_attribute('lower_fov',str(-30))
    lidar_location = carla.Location(0,0,2.5)
    lidar_rotation = carla.Rotation(0,0,0)
    lidar_transform = carla.Transform(lidar_location,lidar_rotation)
    lidar_sen = world.spawn_actor(lidar_bp,lidar_transform,attach_to=Ego_Vehicle)
    #Turn on Lidar and start listening
    lidar_sen.listen(lambda lidar_data: process_lidar_sematic(lidar_data))
    #lidar_sen.listen(lambda point_cloud: point_cloud.save_to_disk('/home/santhanam.17/Carla_scripts/point/%.6d.ply' % point_cloud.frame))
    time.sleep(1)
    #Destro the lidar
    lidar_sen.destroy()

In [ ]:
# --------------
# Add a new LIDAR sensor to my ego
lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast_semantic')
lidar_bp.set_attribute('channels',str(64))
lidar_bp.set_attribute('points_per_second',str(1280000))
lidar_bp.set_attribute('rotation_frequency',str(10))
lidar_bp.set_attribute('range',str(120))
lidar_bp.set_attribute('upper_fov',str(0))
lidar_bp.set_attribute('lower_fov',str(-30))
lidar_location = carla.Location(0,0,2.5)
lidar_rotation = carla.Rotation(0,0,0)
lidar_transform = carla.Transform(lidar_location,lidar_rotation)
lidar_sen = world.spawn_actor(lidar_bp,lidar_transform,attach_to=Ego_Vehicle)
#Turn on Lidar and start listening
#lidar_sen.listen(lambda lidar_data: process_lidar_sematic(lidar_data))
#lidar_sen.listen(lambda point_cloud: point_cloud.save_to_disk('/home/santhanam.17/Carla_scripts/point/%.6d.ply' % point_cloud.frame))
#time.sleep(1)
#Destro the lidar
lidar_sen.destroy()

In [19]:
def merge_ply():
    # assign directory
    directory = '/home/santhanam.17/Carla_scripts/point'
    pcds=np.empty([1,3])

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            pcd1 = o3d.io.read_point_cloud(f)
            point_cloud_in_numpy = np.asarray(pcd1.points)
            pcds =np.append(pcds,point_cloud_in_numpy,axis=0)
            point_cloud = o3d.geometry.PointCloud()
            point_cloud.points = o3d.utility.Vector3dVector(pcds)
            os.remove(f)

    #o3d.io.write_point_cloud("/home/santhanam.17/Carla_scripts/processed/"+str(current_time)+"_Final_Point_cloud.ply", point_cloud,write_ascii=True)
    #final_ply = o3d.io.read_point_cloud("/home/santhanam.17/Carla_scripts/processed/"+str(current_time)+"_sync.ply")
    #o3d.visualization.draw_geometries([final_ply])

    return point_cloud


In [ ]:
transform=Target_Vehicle.get_transform()
key_tuple=(transform.location.x,transform.location.y,transform.location.z\
            ,transform.rotation.pitch,transform.rotation.yaw,transform.rotation.roll)
print(key_tuple)

In [20]:
def pointcloud_gen():
    #Create a loop to update the location of the target vehicle
    Target_Vehicle.set_transform(carla.Transform(carla.Location(x=5,y=0, z=0.598),carla.Rotation(pitch=0.0, yaw=0.0, roll=0.000000)))

    dict={}

    for i in range(5,120,2):
        print(i)
        for j in range(0,360,1):

            transform=Target_Vehicle.get_transform()
            transform.location.x = i*math.sin(math.radians(j))
            transform.location.y = i*math.cos(math.radians(j))
            Target_Vehicle.set_transform(transform)
            key_tuple=(transform.location.x,transform.location.y,transform.location.z\
                ,transform.rotation.pitch,transform.rotation.yaw,transform.rotation.roll)
            start_lidar()
            point_cloud=merge_ply()
            point_cloud_in_numpy = np.asarray(point_cloud.points)
            dict.update({key_tuple:point_cloud_in_numpy})



    with open('/home/santhanam.17/Carla_scripts/point_cloud_database.pickle', 'wb') as handle:
        pickle.dump(dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
pointcloud_gen()

In [ ]:

print(b)

In [ ]:
print(len(dict))

In [23]:
final_ply = o3d.io.read_point_cloud("/home/santhanam.17/Carla_scripts/processed/04:58:25_Final_Point_cloud.ply")
o3d.visualization.draw_geometries([final_ply])


Format = auto
Extension = ply


In [11]:
import time

st = time.time()
with open('/home/santhanam.17/Carla_scripts/filename.pickle', 'rb') as handle:
    read_dict = pickle.load(handle)
et= time.time()
x=list(read_dict.keys())
pcd =read_dict[x[0]]

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(pcd)

elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
#o3d.visualization.draw_geometries([point_cloud])
#print(pcd)

pcd1 = o3d.io.read_point_cloud("/home/santhanam.17/Carla_scripts/processed/06:36:53sync.ply")
point_cloud_in_numpy = np.asarray(pcd1.points)
print(point_cloud_in_numpy)

Execution time: 0.012782096862792969 seconds
[[ 3.97071390e-316  0.00000000e+000  0.00000000e+000]
 [ 1.00306883e+001  2.08309814e-002 -1.00372279e+000]
 [ 9.47924900e+000 -4.39416260e-001 -1.02928650e+000]
 ...
 [ 8.33544540e+000  8.96605670e-001 -2.35875916e+000]
 [ 8.47096920e+000 -7.22042143e-001 -2.46843600e+000]
 [ 8.46991825e+000  7.05733836e-001 -2.46773386e+000]]
Format = auto
Extension = ply


In [7]:
def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

In [24]:

vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=1920,
    height=1080,
    left=50,
    top=50)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True

add_open3d_axis(vis)
for item in read_dict.values():
    print(item)
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(item)
    vis.add_geometry(point_cloud)
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.5)
    vis.remove_geometry(point_cloud)

vis.destroy_window()

[[ 5.87069433e-316  0.00000000e+000  1.53530899e-319]
 [ 8.29287767e-001  5.31376505e+000 -1.27462876e+000]
 [ 7.90887058e-001  5.17466259e+000 -1.24065828e+000]
 ...
 [-1.40145433e+000  4.11184597e+000 -2.46016598e+000]
 [-1.41585326e+000  4.11181021e+000 -2.46278977e+000]
 [-1.43028069e+000  4.11177397e+000 -2.46544313e+000]]
[[ 3.73655445e-316  0.00000000e+000  6.21648314e-310]
 [ 2.66940761e+000  4.61036777e+000 -1.03094780e+000]
 [ 2.65564203e+000  4.62003803e+000 -1.03123772e+000]
 ...
 [ 2.55001247e-001  3.68170667e+000 -2.13072562e+000]
 [ 2.44221359e-001  3.69403982e+000 -2.13741088e+000]
 [ 2.34267280e-001  3.72065639e+000 -2.15237570e+000]]
[[ 3.25518906e-316  0.00000000e+000  6.21648314e-310]
 [ 2.12946010e+000  3.14061522e+000 -1.52573717e+000]
 [ 2.04795337e+000  3.02040553e+000 -1.50268984e+000]
 ...
 [ 1.91829109e+000  2.86595249e+000 -1.99110591e+000]
 [ 1.93591285e+000  2.91202211e+000 -2.01887989e+000]
 [ 1.96211529e+000  2.97163391e+000 -2.05592608e+000]]
[[ 4.28098